In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.loaders import Sentinel2OdcLoader
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsWriter
from pystac import Item

from grid import grid

from run_task import CloudlessProcessor

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
# Study site configuration
region_code = "FJ"
region_index = "63,20"
datetime = "2023-09"
dep_path = LocalPath(
    local_folder="data",
    sensor="s2",
    dataset_id="cloudless",
    version="0.0.0",
    time=datetime,
)
item_id = f"{region_code}_{region_index}"

# Set up a data loader
loader = Sentinel2OdcLoader(
    epsg=3857,
    datetime=datetime,
    dask_chunksize=dict(band=1, time=1, x=4096, y=4096),
    odc_load_kwargs=dict(
        fail_on_error=False, resolution=10, bands=["SCL", "B04", "B03", "B02"]
    ),
)

# And a data processor
processor = CloudlessProcessor()

# And get the study site
cell = grid.loc[[(region_index, region_code)]]
cell.explore()

In [ ]:
# Run the load process, which uses Dask, so it's fast
input_data = loader.load(cell)
input_data

In [ ]:
# Load data into memory. This will take 5-10 minutes or so.
output_data = processor.process(input_data).compute()
output_data

In [ ]:
# Plot data.
output_data.plot.imshow(robust=True)

In [ ]:
writer = LocalDsWriter(itempath=dep_path)
out_files = writer.write(output_data, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()